In [15]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
import torch.nn.functional as F  
import torchvision.models as models  # Import this line to access pre-trained models
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc


In [28]:
import os
import numpy as np
import pandas as pd

def merge_files_with_limit(directory, output_file, max_files, prefix="W2V_DF_hidden_states_features"):
    """
    Merge a specific number of .npy or .csv files based on availability.

    Parameters:
        directory (str): Directory containing the files.
        output_file (str): Path to save the merged file.
        max_files (int): Maximum number of files to merge.
        prefix (str): File prefix to search for.

    Returns:
        pd.DataFrame: Merged data.
    """
    # Identify .npy files with the specified prefix
    npy_files = [os.path.join(directory, f) for f in os.listdir(directory)
                 if f.startswith(prefix) and f.endswith('.npy')]
    
    if npy_files:
        print("Found .npy files. Loading and merging...")
        # Limit the number of files to merge
        npy_files = npy_files[:max_files]
        # Load and concatenate the limited number of .npy files
        data_list = [np.load(file) for file in npy_files]
        
        # Reshape the 3D data to 2D: (samples * segments, features)
        reshaped_list = [data.reshape(-1, data.shape[-1]) for data in data_list]
        merged_data = np.vstack(reshaped_list)
        
        # Convert to DataFrame for saving
        merged_df = pd.DataFrame(merged_data)
        merged_df.to_csv(output_file, index=False)
        return merged_df
    else:
        print(".npy files not found. Checking for .csv files...")
        # Identify .csv files with the specified prefix
        csv_files = [os.path.join(directory, f) for f in os.listdir(directory) 
                     if f.startswith(prefix) and f.endswith('.csv')]
        
        if not csv_files:
            raise FileNotFoundError("No .npy or .csv files found with the specified prefix.")
        
        # Limit the number of files to merge
        csv_files = csv_files[:max_files]
        # Load and concatenate the limited number of .csv files
        df_list = [pd.read_csv(file) for file in csv_files]
        merged_df = pd.concat(df_list, ignore_index=True)
        # Save the merged DataFrame to a single CSV file
        merged_df.to_csv(output_file, index=False)
        return merged_df

# Directory and output file paths
csv_dir = r"F:\Awais_data\Datasets\ASV21\Features\SSL\W2V\DF\Hidden"
output_file = "merged_features_ASV21.csv"
max_files_to_merge = 1  # Specify the number of files to merge

# Merge the files
merged_data = merge_files_with_limit(csv_dir, output_file, max_files_to_merge)

# Verify the result
print(f"Merged data shape: {merged_data.shape}")


Found .npy files. Loading and merging...


KeyboardInterrupt: 

In [16]:
# --------- 2. Preprocess Data ---------
def preprocess_data(df):
    # Extract features and labels
    file_names = df.iloc[:, 0]  # File names (not used for training)
    labels = df.iloc[:, -1].values  # Labels (1=real, 0=fake)
    features = df.iloc[:, 1:-1].values  # Features (1x1084)
    
    # Normalize features
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features)
    
    # Split data
    X_train, X_temp, y_train, y_temp = train_test_split(features_normalized, labels, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    return X_train, X_val, X_test, y_train, y_val, y_test, features_normalized, labels

X_train, X_val, X_test, y_train, y_val, y_test, features, labels = preprocess_data(merged_data)

In [17]:
# --------- 3. Dataset and Dataloader ---------
class AudioDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create DataLoaders
train_dataset = AudioDataset(X_train, y_train)
val_dataset = AudioDataset(X_val, y_val)
test_dataset = AudioDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [22]:
class AttentionModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, num_classes=2, num_heads=8):
        super(AttentionModel, self).__init__()

        # Embedding layer to transform input features into the desired embedding dimension
        self.embedding = nn.Linear(input_dim, embedding_dim)

        # Multi-Head Self Attention Layer
        self.attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads, batch_first=True)

        # Feed Forward Network after Attention
        self.fc1 = nn.Linear(embedding_dim, embedding_dim * 2)
        self.fc2 = nn.Linear(embedding_dim * 2, num_classes)

        # Layer Normalization for stability
        self.layer_norm = nn.LayerNorm(embedding_dim)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Embedding input to match the attention input size
        x = self.embedding(x)  # [batch_size, seq_len, embedding_dim]

        # Adding a batch dimension for attention mechanism
        x = x.unsqueeze(1)  # Add a sequence length dimension [batch_size, 1, embedding_dim]

        # Attention Layer (self-attention)
        attn_output, attn_weights = self.attention(x, x, x)  # (batch_size, seq_len, embedding_dim), (batch_size, seq_len, seq_len)
        
        # Layer normalization after attention
        x = self.layer_norm(attn_output + x)

        # Feed Forward Network
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x.squeeze(1), attn_weights  # Squeeze to get batch_size, num_classes output

input_dim = 512
embedding_dim = 128
num_classes = 2
# Instantiate the model
model = AttentionModel(input_dim=input_dim, embedding_dim=embedding_dim, num_classes=num_classes).cuda()


In [23]:
# --------- NT-Xent Loss ---------
class NTXentLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(NTXentLoss, self).__init__()
        self.temperature = temperature
    
    def forward(self, z_i, z_j):
        z_i = nn.functional.normalize(z_i, dim=1)
        z_j = nn.functional.normalize(z_j, dim=1)
        sim_matrix = torch.mm(z_i, z_j.t()) / self.temperature
        labels = torch.arange(z_i.size(0)).cuda()
        loss = nn.CrossEntropyLoss()(sim_matrix, labels)
        return loss


criterion = NTXentLoss()

In [24]:
 #--------- 6. Optimizer ---------
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [25]:
def calculate_eer(fpr, tpr):
    # Calculate EER (Equal Error Rate) from the FPR and TPR
    eer_index = np.nanargmin(np.abs(fpr - (1 - tpr)))  # Find where the FPR and TPR are closest
    eer = (fpr[eer_index] + (1 - tpr[eer_index])) / 2
    return eer

def evaluate_model(model, val_loader, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []
    
    with torch.no_grad():  # No gradients needed for evaluation
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)

            logits, _ = model(x)  # Forward pass
            preds = torch.softmax(logits, dim=1)[:, 1]  # Assuming binary classification, take the positive class
            all_labels.extend(y.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    # Calculate AUC
    auc_score = roc_auc_score(all_labels, all_preds)

    # Calculate Confusion Matrix
    cm = confusion_matrix(all_labels, (all_preds > 0.5).astype(int))

    # Compute ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(all_labels, all_preds)
    eer = calculate_eer(fpr, tpr)

    return auc_score, cm, eer

def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def train_contrastive_model(model, criterion, optimizer, train_loader, val_loader=None, epochs=50):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            # Forward pass
            logits, _ = model(x)

            # Compute loss
            loss = nn.CrossEntropyLoss()(logits, y)  # Using cross-entropy for classification
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

        if val_loader:
            # Evaluate model on validation set after each epoch
            auc_score, cm, eer = evaluate_model(model, val_loader, device)
            print(f"Validation AUC: {auc_score:.4f}, EER: {eer:.4f}")

            # Confusion Matrix Plot
            # plot_confusion_matrix(cm, classes=["Fake", "Real"], title=f'Confusion Matrix Epoch {epoch+1}')
            # plt.savefig(f'confusion_matrix_epoch_{epoch+1}.png')
            # plt.close()

# Training the model with validation evaluation
train_contrastive_model(model, criterion, optimizer, train_loader, val_loader, epochs=100)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x510 and 512x128)

In [ ]:
# --------- 8. t-SNE Visualization ---------
def plot_tsne(features, labels, title, save_path):
    tsne = TSNE(n_components=2, random_state=42)
    tsne_features = tsne.fit_transform(features)
    plt.figure(figsize=(8, 8))
    plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=labels, cmap='viridis', s=10)
    plt.colorbar()
    plt.title(title)
    plt.savefig(save_path)
    plt.show()

# t-SNE before training
plot_tsne(features[:80000], labels[:80000], "t-SNE Before Training", "tsne_before.png")

In [ ]:
# t-SNE after training
model.eval()

In [ ]:
# Use torch.float32 dtype for input
with torch.no_grad():
    final_embeddings, _ = model(torch.tensor(features[:80000], dtype=torch.float32).cuda())  # Get only the embeddings (ignore attention_weights)
    final_embeddings = final_embeddings.cpu().numpy()  # Convert embeddings to CPU for visualization

plot_tsne(final_embeddings, labels[:80000], "t-SNE After Training", "tsne_after.png")



In [ ]:
print(model)

In [ ]:
def model_summary(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total Parameters: {total_params}")
    for name, param in model.named_parameters():
        print(f"{name}: {param.size()}")

model_summary(model)

In [ ]:
pwd

In [ ]:
# Saving the model
# Define local directory to save the model

save_directory = r"C:\Notebooks\rrl_source\Spectnet_model_ASV21_embedding"  # Replace with your preferred path
os.makedirs(save_directory, exist_ok=True)  # Create directory if it doesn't exist

# Define file paths for model and weights
model_path = os.path.join(save_directory, "specnet_ASV21_model.pth")
weights_path = os.path.join(save_directory, "specnet_ASV21_weights.pth")

# Save the trained SpecNet model and weights
torch.save(model.state_dict(), weights_path)  # Save weights
torch.save(model, model_path)                # Save the entire model

print(f"Trained model saved at: {model_path}")
print(f"Trained model weights saved at: {weights_path}")

In [ ]:
# Loading the model
# Define the paths to the saved model and weights
os.chdir(r'C:\Notebooks\rrl_source\Spectnet_model_ASV21_embedding')

model_path = ".\specnet_Halftruth_model.pth"  # Replace with your saved model path
weights_path = ".\specnet_Halftruth_weights.pth"  # Replace with your weights path

def load_model_from_local(model_path, weights_path, device="cuda"):
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    
    # Load the full model
    loaded_model = torch.load(model_path, map_location=device)
    loaded_model.eval()  # Set the model to evaluation mode
    print("Model and weights successfully loaded from local directory.")
    return loaded_model

# Load the model
device = "cuda"  # Specify the device to load the model
loaded_model = load_model_from_local(model_path, weights_path, device)


In [26]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define a custom dataset
class FeatureDataset(Dataset):
    def __init__(self, features):
        self.features = torch.tensor(features, dtype=torch.float32)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx]

# Function to extract embeddings
def extract_embeddings(features, model, device="cuda"):
    model.eval()  # Set the model to evaluation mode
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    model.to(device)

    dataset = FeatureDataset(features)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

    embeddings_128 = []
    embeddings_256 = []

    with torch.no_grad():  # No gradients required for inference
        for batch in dataloader:
            batch = batch.to(device)
            # Forward pass through the model
            embedded = model.embedding(batch)  # Output of the embedding layer (128 features)
            attention_output, _ = model.attention(embedded.unsqueeze(1), embedded.unsqueeze(1), embedded.unsqueeze(1))
            normed_output = model.layer_norm(attention_output.squeeze(1))  # Output of LayerNorm (128 features)
            embeddings_128.append(normed_output.cpu().numpy())
            
            fc1_output = model.fc1(normed_output)  # Output of fc1 (256 features)
            embeddings_256.append(fc1_output.cpu().numpy())
    
    return np.vstack(embeddings_128), np.vstack(embeddings_256)

# Main function
def process_features_and_save_embeddings(input_csv, output_csv_128, output_csv_256, model, device="cuda"):
    # Load the feature data
    data = pd.read_csv(input_csv)
    
    # Extract necessary columns
    file_ids = data.iloc[:, 0].values  # FileID
    features = data.iloc[:, 1:-1].values  # Features
    labels = data.iloc[:, -1].values  # Label

    # Extract embeddings
    embeddings_128, embeddings_256 = extract_embeddings(features, model, device)

    # Create DataFrames for 128 and 256 features
    embedding_columns_128 = [f"Feature{i+1}" for i in range(embeddings_128.shape[1])]
    df_embeddings_128 = pd.DataFrame(embeddings_128, columns=embedding_columns_128)
    df_embeddings_128.insert(0, "FileID", file_ids)
    df_embeddings_128["Label"] = labels

    embedding_columns_256 = [f"Feature{i+1}" for i in range(embeddings_256.shape[1])]
    df_embeddings_256 = pd.DataFrame(embeddings_256, columns=embedding_columns_256)
    df_embeddings_256.insert(0, "FileID", file_ids)
    df_embeddings_256["Label"] = labels

    # Save the updated DataFrames to CSV files
    df_embeddings_128.to_csv(output_csv_128, index=False)
    df_embeddings_256.to_csv(output_csv_256, index=False)
    print(f"Embeddings saved to {output_csv_128} and {output_csv_256}")

# Paths
input_csv = r"C:\Notebooks\rrl_source\Spectnet_model_ASV21_embedding\features.csv"  # Input file
output_csv_128 = r"C:\Notebooks\rrl_source\Spectnet_model_ASV21_embedding\feature_emb_128.csv"  # 128 features file
output_csv_256 = r"C:\Notebooks\rrl_source\Spectnet_model_ASV21_embedding\feature_emb_256.csv"  # 256 features file
model_path = r".\specnet_Halftruth_model.pth"  # Model path
device = "cuda"

# Load the model
def load_model(model_path, device):
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path, map_location=device)
    model.eval()
    print("Model loaded successfully.")
    return model

loaded_model = load_model(model_path, device)

# Process the CSV and save embeddings
process_features_and_save_embeddings(input_csv, output_csv_128, output_csv_256, loaded_model, device)


C:\Users\mawais\AppData\Local\Temp\ipykernel_27548\126609002.py:83: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


FileNotFoundError: [Errno 2] No such file or directory: '.\\specnet_Halftruth_model.pth'